In [ ]:
import os, sys
from pathlib import Path
from typing import Dict, Tuple, List
import spacy

In [ ]:

HOME = os.getcwd()
current = HOME 
while 'src' not in os.listdir(current):
    current = Path(current).parent

PARENT_DIR = current

DATA_FOLDER = os.path.join(PARENT_DIR, 'src','data')
data_path = os.path.join(DATA_FOLDER, 'filtered.tsv')

sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'data_analysis'))
sys.path.append(os.path.join(str(current), 'evaluation'))
sys.path.append(os.path.join(str(current), 'text_processing'))

In [ ]:
# let's first fix the data 
import src.text_processing.preprocess as pr
import src.data_preparation.prepare_data as prd 
# fixed_data = prd.fix_initial_data(data_path)
data = prd.prepare_all_data(fixed_data_file=os.path.join(DATA_FOLDER, 'fixed.csv'), save=False)

In [ ]:
import random
def process_text(text: str) -> str:
    return pr.no_extra_spaces(pr.no_extra_chars(pr.to_lower(text)))

def process_batch(batch: Dict, nlp):
    p = random.random()
    if p < 10 ** -5:
        print("really ?")
    return dict([(k, [process_text(t) for t in v]) for k, v in ({"source": pr.uniform_ne_batched(batch['source'], nlp), 
                                                                 "target": pr.uniform_ne_batched(batch['target'], nlp)}).items()]) 

# load the nlp object in advnace
nlp = spacy.load("en_core_web_sm")
processed_data = data.map(lambda b: process_batch(b, nlp), batched=True)

In [ ]:
# processed_data = processed_data.filter(lambda s: (isinstance(s['source'], str) and isinstance(s['target'], str)))
# # save the data
# processed_data.to_csv(os.path.join(DATA_FOLDER, 'all_data_processed.csv'), index=False)

In [ ]:
from datasets import load_dataset
processed_data = load_dataset("csv", data_files=os.path.join(DATA_FOLDER, "all_data_processed.csv"), split='train')
processed_data = processed_data.filter(lambda s: (isinstance(s['source'], str) and isinstance(s['target'], str)))
processed_data

In [ ]:
# let's take a small smaple of 10000 rows
from src.toxicity_scores import n_grams as ng
import importlib
importlib.reload(ng)
# sample = processed_data.select(range(10 ** 4))
uni, bi = ng.build_unigram_counter(processed_data)
ignore_map = ng.build_ignore_toxic_map(0.1, pr.standard_stop_words())
# ignore_map

In [ ]:
u, b = dict([(k, (v["source"] + 1) / (v["target"] + 1)) for k, v in uni.items()]) , dict([(k, (v["source"] + 1) / (v["target"] + 1)) for k, v in bi.items()]) 

In [ ]:
from empiricaldist import Cdf

u_scores = [v for k, v in u.items()]
bi_scores = [v for k, v in b.items()]

cdf_u = Cdf.from_seq(u_scores)
cdf_b = Cdf.from_seq(bi_scores)

toxicity_threshold_u = cdf_u.forward(0.4).item()
toxicity_threshold_bi = cdf_b.forward(0.4).item()
default_toxicitiy = cdf_u.forward(0.2).item()
toxicity_threshold_bi, toxicity_threshold_u, default_toxicitiy


In [ ]:
def mask_sentence(s: str, masks: List[str], mask_token):
    ls = pr.lemmatize(s)
    return " ".join([(c if c not in masks else mask_token) for c in ls])

def restored_sentences(masted_sentence: str, words: List[str], mas):
    ls = pr.lemmatize(masted_sentence)

    return " ".join([(c if c not in masks else mask_token) for c in ls])
        


def baseline_predict(sentences : str, 
                     masked_ml_pipe, 
                     uni_gram, 
                     bi_gram, 
                     mask_token: int ):
    # first extract the toxicity attribued
    masks = [ng.get_toxicity_attributes(s, 
                                    uni_gram=uni_gram, 
                                    bi_gram=bi_gram) for s in sentences]
    
    masked_sentences = [mask_sentence(s, m, mask_token) for s, m in zip(sentences)]

    # pass the results through the pipline
    model_outputs = masked_ml_pipe(masked_sentences, top_k=1)

    #

In [ ]:
importlib.reload(ng)
s = "I hate your face !!"  
res = ng.get_toxicity_attributes(s, 
                                 uni_threshold=toxicity_threshold_u, 
                                 bi_threshold=toxicity_threshold_bi, 
                                 uni_gram=u, 
                                 bi_gram=b, 
                                 default_toxicity=default_toxicitiy)


print(res)

In [ ]:
ls = pr.lemmatize(s)
# replace them with mask tokens
# pass it to a pretrained model.
# masked sentence 
    

In [ ]:
def pipelin 

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline
ckpnt = "distilbert-base-uncased"

mask_filler = pipeline("fill-mask", ckpnt)
# bert = AutoModelForMaskedLM.from_pretrained(ckpnt)
bert_tokenizer = AutoTokenizer.from_pretrained(ckpnt)
bert_tokenizer.mask_token

In [ ]:
masked_ls = " ".join([(c if c not in res else bert_tokenizer.mask_token) for c in ls])
masked_ls
output = mask_filler([masked_ls, masked_ls], top_k=1)
output[0]

In [ ]:
import pickle
with open(os.path.join(PARENT_DIR, 'src', 'toxicity_scores', 'uni_gram.pk'), 'rb') as f:
    counter_loaded = pickle.load(f) 